In [1]:
import pandas as pd
import json
import numpy as np
from types import NoneType
import re
from IPython.display import display, Markdown

# !pip install mlxtend
# from mlxtend.preprocessing import TransactionEncoder
# from mlxtend.frequent_patterns import apriori, association_rules
import ast
import json
from collections import defaultdict
from itertools import combinations, chain, product

from google import genai
from google.genai import types
import os
os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")

In [2]:
features_df = pd.read_parquet(
    'data/content-features-dataset/content_features-new-with-stats-top50WeeklyVideos.parquet')
# features_df = pd.read_csv('content_features-raw-top50WeeklyVideos.csv')

# features_df = features_df[features_df['categories']
#                           != 'Không liên quan ẩm thực']

In [3]:
features_df['year'] = features_df['createTime'].dt.isocalendar().year
features_df['week'] = features_df['createTime'].dt.isocalendar().week

In [4]:
features_df['engagement_rate'] = (features_df['statsV2.diggCount'] + features_df['statsV2.commentCount'] +
                                  features_df['statsV2.shareCount'] + features_df['statsV2.collectCount']) / features_df['statsV2.playCount']

In [5]:
features_df['transcript_len'] = features_df['transcript'].apply(len)
features_df['cta_count'] = features_df['cta_content'].apply(len)
# features_df = features_df[features_df['transcript_len'] > 0]

features_df['transcript_word_count'] = features_df['transcript'].str.split().str.len()

In [6]:
mean_word_per_second = features_df['transcript_word_count'].mean(
) / features_df['video.duration'].mean()

features_df.loc[features_df['video.duration'] == 0,
                'video.duration'] = features_df['transcript_word_count'] / mean_word_per_second
features_df['word_per_second'] = features_df['transcript_word_count'] / \
    features_df['video.duration']
features_df['word_per_second'] = features_df['word_per_second'].fillna(0)

# features_df = features_df[features_df['word_per_second'] < 8]

In [7]:
def remove_hashtags(text):
    return re.sub(r'\s*#\w+', '', text).strip()


# Tạo cột mới: desc không có hashtag
features_df['desc_clean'] = features_df['desc'].apply(remove_hashtags)

# Tạo cột đếm số từ trong desc_clean
features_df['desc_word_count'] = features_df['desc_clean'].apply(
    lambda x: len(x.split()) if isinstance(x, str) else 0)

In [8]:
def parse_list(x):
    if isinstance(x, np.ndarray):
        return list(x)
    if isinstance(x, NoneType):
        return []


for col in features_df.columns:
    str_flag = features_df[col].apply(
        lambda x: isinstance(x, np.ndarray)).any()
    if str_flag:
        features_df[col] = features_df[col].apply(parse_list)

In [9]:
full_df = pd.read_parquet(
    "data/content-features-dataset/preprocessed_videos.parquet")
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70996 entries, 0 to 70995
Data columns (total 52 columns):
 #   Column                           Non-Null Count  Dtype                           
---  ------                           --------------  -----                           
 0   CategoryType                     70996 non-null  object                          
 1   author.downloadSetting           70996 non-null  object                          
 2   author.duetSetting               70996 non-null  object                          
 3   author.id                        70996 non-null  object                          
 4   author.nickname                  70996 non-null  object                          
 5   author.openFavorite              70996 non-null  bool                            
 6   author.secUid                    70996 non-null  object                          
 7   author.signature                 70996 non-null  object                          
 8   author.stitchSet

In [10]:
full_df_to_merge = full_df[['video.id', 'author.uniqueId']]

In [11]:
fnial_df = features_df.merge(full_df_to_merge, how='left', on='video.id')

In [12]:
fnial_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3004 entries, 0 to 3003
Data columns (total 46 columns):
 #   Column                      Non-Null Count  Dtype                           
---  ------                      --------------  -----                           
 0   categories                  3004 non-null   object                          
 1   audience_target             3004 non-null   object                          
 2   body_analysis               3004 non-null   object                          
 3   content_style               3004 non-null   object                          
 4   cta_content                 3004 non-null   object                          
 5   cta_type                    3004 non-null   object                          
 6   emotion_keywords            3004 non-null   object                          
 7   has_cta                     3004 non-null   bool                            
 8   highlight_sentences         3004 non-null   object                  

In [ ]:
explode = fnial_df.explode('hashtags')

explode[explode['hashtags'] == 'thithuthao'][['hashtags', 'author.uniqueId']]

,hashtags,author.uniqueId
3,thithuthao,donghyeontv
16,thithuthao,donghyeontv
17,thithuthao,donghyeontv
31,thithuthao,donghyeontv
34,thithuthao,donghyeontv
...,...,...
2976,thithuthao,donghyeontv
2987,thithuthao,donghyeontv
2988,thithuthao,donghyeontv
2990,thithuthao,donghyeontv


In [13]:
# fnial_df.to_parquet(
#     'data/content_analysis/content_analysis_data.parquet', index=False)